# Custom Pipelines in EvalML

EvalML pipelines consist of modular components combining any number of transformers and an estimator. This allows you to create pipelines that fit the needs of your data to achieve the best results.

## Requirements
A custom pipeline must adhere to the following requirements:

1. Inherit from the proper pipeline base class
    - Multiclass classification - `MulticlassClassificationPipeline`
    - Binary classification - `BinaryClassificationPipeline`
    - Regression - `RegressionPipeline`


2. Have a `component_graph` list as a class variable detailing the structure of the pipeline
    - components can be passed in as string representations or component objects

## Addtional configuration
There are a few other options to configure your custom pipeline.

### Custom Name
By default, your custom pipeline's `name` property will be the seperated class name at each capitalized word. Therefore, we require pipeline class names to be in Pascal case. However you can override this behavior by provided a `custom_name` class attribute to your pipeline.
    
### Custom Hyperparameters
If you intend on using custom pipelines with AutoML, we provide a `custom_hyperparameters` attribute for more you to specify the search space for this pipeline's hyperparameters. By default, EvalML pulls hyperparameter ranges that is detailed on each component. 

To apply custom hyperparameter ranges, `custom_hyperparameters` will be a dictionary where each key value pair is the parameter and the parameter range. This will override the default behavior for each specified parameter in the `hyperparameters` property.

Examples for both will be below.

In [ ]:
from evalml.pipelines import MulticlassClassificationPipeline
from evalml.pipelines.components import StandardScaler, SimpleImputer
from evalml.pipelines.components.estimators import LogisticRegressionClassifier


# the pipeline needs to be a subclass of one of our base pipelines, in this case `MulticlassClassificationPipeline`
class CustomPipeline(MulticlassClassificationPipeline):    
    # components can be passed in as objects or as component name strings
    component_graph = ['Simple Imputer', StandardScaler(), 'Logistic Regression Classifier']
    
    # custom_name overrides default name behavior
    custom_name = "New Classification Pipeline"
    
    # you can override component hyperparameter_ranges like so
    # ranges must adhere to skopt tuner
    custom_hyperparameters = {
        "impute_strategy":["most_frequent"]
    }

In [ ]:
CustomPipeline.name

In [ ]:
CustomPipeline.hyperparameters

In [ ]:
# a parameters dictionary is necessary to instantiate pipelines
parameters = {
    'Simple Imputer':{
        'impute_strategy':"most_frequent"
    },
    'Logistic Regression Classifier':{
        'penalty':'l2',
        'C':5,
    }
}

pipeline = CustomPipeline(parameters=parameters, random_state=3)

In [ ]:
from evalml.demos import load_wine

X, y = load_wine()

pipeline.fit(X, y)
pipeline.score(X, y, ['recall_micro'])